In [1]:
import numpy as np
import pandas as pd
import datetime
import math
from os import listdir
from os.path import isfile, join
import json

In [26]:
out='../output/'
inb='../input/flows/'
pref='dam'
inp = inb + pref + '/'

In [38]:
dat=pd.DataFrame(columns=['date','flow','from','hour','to'])
onlyfiles = [f for f in listdir(inp ) if isfile(join(inp, f))]
for f in onlyfiles:
    if (f.split('_')[0]==pref):
        #print(f)
        with open(join(inp, f), "r") as read_file:
            data = pd.DataFrame(json.load(read_file))
            dat=pd.concat([dat,data])

dat['flow']=dat['flow'].astype(int)
dat

,date,flow,from,hour,to
0,2016-01-01,160,ZAMZ,0,ZAMC
1,2016-01-01,170,ZIM,0,ZAMZ
2,2016-01-01,160,ZAMZ,1,ZAMC
3,2016-01-01,170,ZIM,1,ZAMZ
4,2016-01-01,160,ZAMZ,2,ZAMC
...,...,...,...,...,...
2494,2016-05-31,10,MOZS,23,RSAN
2495,2016-05-31,10,RSAN,23,BOT
2496,2016-05-31,10,ZAMZ,23,ZAMC
2497,2016-05-31,10,ZIM,23,ZAMZ


In [39]:
dam = pd.read_csv(out + pref + '_hourly.csv')
dam=dam.set_index([pd.DatetimeIndex(dam['date']),'hour'])
dam=dam.drop(['date'],1)
dam

price  volume  year  month  day  day_of_week      value  \
date       hour                                                             
2016-01-01 0      40.10   477.5  2016      1    1            4  19147.750   
           1      40.08   486.6  2016      1    1            4  19502.928   
           2      40.06   495.6  2016      1    1            4  19853.736   
           3      40.08   496.6  2016      1    1            4  19903.728   
           4      40.07   502.9  2016      1    1            4  20151.203   
...                 ...     ...   ...    ...  ...          ...        ...   
2021-05-07 19    294.98    60.0  2021      5    7            4  17698.800   
           20    124.05    60.0  2021      5    7            4   7443.000   
           21     86.15    78.0  2021      5    7            4   6719.700   
           22     46.18    72.2  2021      5    7            4   3334.196   
           23     43.96    66.8  2021      5    7            4   2936.528   

                      cat  
date       hour            
2016-01-01 0          off  
           1          off  
           2          off  
           3          off  
           4          off  
...                   ...  
2021-05-07 19     evening  
           20    standard  
           21    standard  
           22         off  
           23         off  

[46020 rows x 8 columns]

In [40]:
flow=dat
#flow['date']=pd.to_datetime(flow['flowdate'],format="%d/%m/%Y") #"%Y-%m-%d")#"%d/%m/%Y")
#flow=flow.drop(['flowdate','flowid','market'],axis=1)

flow=flow.set_index([pd.DatetimeIndex(flow['date'])])
flow['year']=flow.index.year
flow['month']=flow.index.month
flow['day']=flow.index.day
flow['day_of_week']=flow.index.dayofweek

flow=flow.set_index([pd.DatetimeIndex(flow['date']),'hour'])
flow['price']=dam['price']
flow['value']=flow['price']*flow['flow']
flow=flow.drop(['date'],axis=1)
flow=flow.dropna(0)
flow=flow.reset_index()

flow

,date,hour,flow,from,to,year,month,day,day_of_week,price,value
0,2016-01-01,0,160,ZAMZ,ZAMC,2016,1,1,4,40.10,6416.0
1,2016-01-01,0,170,ZIM,ZAMZ,2016,1,1,4,40.10,6817.0
2,2016-01-01,1,160,ZAMZ,ZAMC,2016,1,1,4,40.08,6412.8
3,2016-01-01,1,170,ZIM,ZAMZ,2016,1,1,4,40.08,6813.6
4,2016-01-01,2,160,ZAMZ,ZAMC,2016,1,1,4,40.06,6409.6
...,...,...,...,...,...,...,...,...,...,...,...
15685,2016-05-31,23,10,MOZS,RSAN,2016,5,31,1,50.85,508.5
15686,2016-05-31,23,10,RSAN,BOT,2016,5,31,1,50.85,508.5
15687,2016-05-31,23,10,ZAMZ,ZAMC,2016,5,31,1,50.85,508.5
15688,2016-05-31,23,10,ZIM,ZAMZ,2016,5,31,1,50.85,508.5


In [46]:
f_to=flow[['date','hour','flow','value','price','to','year','month','day','day_of_week']].rename(columns={'to': 'node'})
f_from=flow[['date','hour','flow','value','price','from','year','month','day','day_of_week']].rename(columns={'from': 'node'})
f_from['flow']=-f_from['flow']
f_from['value']=-f_from['value']

flows=pd.concat([f_to, f_from])

flows

,date,hour,flow,value,price,node,year,month,day,day_of_week
0,2016-01-01,0,160,6416.0,40.10,ZAMC,2016,1,1,4
1,2016-01-01,0,170,6817.0,40.10,ZAMZ,2016,1,1,4
2,2016-01-01,1,160,6412.8,40.08,ZAMC,2016,1,1,4
3,2016-01-01,1,170,6813.6,40.08,ZAMZ,2016,1,1,4
4,2016-01-01,2,160,6409.6,40.06,ZAMC,2016,1,1,4
...,...,...,...,...,...,...,...,...,...,...
15685,2016-05-31,23,-10,-508.5,50.85,MOZS,2016,5,31,1
15686,2016-05-31,23,-10,-508.5,50.85,RSAN,2016,5,31,1
15687,2016-05-31,23,-10,-508.5,50.85,ZAMZ,2016,5,31,1
15688,2016-05-31,23,-10,-508.5,50.85,ZIM,2016,5,31,1


In [47]:
flows.loc[flows['node']=='ZIMA','node']='ZIM'
flows.loc[flows['node']=='MOZN_EDM','node']='MOZ'
flows.loc[flows['node']=='MOZN_HCB','node']='HCB'
flows.loc[flows['node']=='MOZS','node']='MOZ'
flows.loc[flows['node']=='RSAS','node']='RSA'
flows.loc[flows['node']=='RSAN','node']='RSA'

In [48]:
flows=flows.reset_index().groupby(['date','hour','node','year','month','day','day_of_week']).sum().reset_index()
flows=flows.loc[flows['flow']!=0]
flows=flows.set_index(['date','hour'])
flows['cat']=dam['cat']
flows.reset_index()
flows

node  year  month  day  day_of_week  index  flow   value  \
date       hour                                                             
2016-01-01 0     ZAMC  2016      1    1            4      0   160  6416.0   
           0     ZAMZ  2016      1    1            4      1    10   401.0   
           0      ZIM  2016      1    1            4      1  -170 -6817.0   
           1     ZAMC  2016      1    1            4      2   160  6412.8   
           1     ZAMZ  2016      1    1            4      5    10   400.8   
...               ...   ...    ...  ...          ...    ...   ...     ...   
2016-05-31 20    ZAMZ  2016      5   31            1  31353    70  9041.9   
           21     HCB  2016      5   31            1  15680   -30 -2035.8   
           21    ZAMC  2016      5   31            1  15681    30  2035.8   
           23     MOZ  2016      5   31            1  15685   -10  -508.5   
           23    ZAMC  2016      5   31            1  15687    10   508.5   

                  price       cat  
date       hour                    
2016-01-01 0      40.10       off  
           0      80.20       off  
           0      40.10       off  
           1      40.08       off  
           1      80.16       off  
...                 ...       ...  
2016-05-31 20    258.34  standard  
           21     67.86  standard  
           21     67.86  standard  
           23     50.85       off  
           23     50.85       off  

[8344 rows x 10 columns]

In [49]:
flowsb=flows.copy()
flowsb.loc[flowsb['flow']<0,'side']='supply'
flowsb.loc[flowsb['flow']>0,'side']='consume'


flow_monthly_cat=flowsb.groupby(['year','month', 'node','cat','side']).sum()
flow_monthly_cat.rename(columns={'flow': 'flow_sum'}, inplace=True)
flow_monthly_cat['hour_count']=flowsb.groupby(['year','month', 'node','cat','side']).count()['flow']
flow_monthly_cat['price_weight_mean'] = flow_monthly_cat['value'] / flow_monthly_cat['flow_sum']
flow_monthly_cat['price_mean']=flowsb.groupby(['year','month', 'node','cat','side']).mean()['price']
flow_monthly_cat=flow_monthly_cat.drop(['day','day_of_week','index','price'],axis=1)
flow_monthly_cat=flow_monthly_cat.loc[flow_monthly_cat['flow_sum']!=0]
flow_monthly_cat['power']=flow_monthly_cat['flow_sum']/flow_monthly_cat['hour_count']
flow_monthly_cat

flow_sum      value  hour_count  \
year month node cat      side                                       
2016 1     BOT  evening  supply        -25   -1501.40           2   
                morning  supply       -285  -14572.40           4   
                off      supply      -1270  -59474.14          45   
                standard supply      -1165  -57809.40          18   
           HCB  evening  supply        -57   -6204.59           2   
...                                    ...        ...         ...   
     5     ZAMZ standard consume      1373  165328.58          29   
           ZIM  morning  consume        74    7995.66           2   
                off      consume       786   43608.85          22   
                         supply       -754  -20435.65          22   
                standard consume       256   22190.93          12   

                                  price_weight_mean  price_mean      power  
year month node cat      side                                               
2016 1     BOT  evening  supply           60.056000   60.050000 -12.500000  
                morning  supply           51.131228   55.290000 -71.250000  
                off      supply           46.830031   48.463556 -28.222222  
                standard supply           49.621803   56.768333 -64.722222  
           HCB  evening  supply          108.852456   99.140000 -28.500000  
...                                             ...         ...        ...  
     5     ZAMZ standard consume         120.414115  182.349310  47.344828  
           ZIM  morning  consume         108.049459  324.195000  37.000000  
                off      consume          55.481997  218.690455  35.727273  
                         supply           27.102984   80.187727 -34.272727  
                standard consume          86.683320  220.971667  21.333333  

[135 rows x 6 columns]

In [50]:
flow_monthly_cat.to_csv(out + pref + '_flow_monthly_cat.csv')